In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

output_correct, output_error = pd.DataFrame(), pd.DataFrame()
    
# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./1.type.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Type No'] = input_['Type No'].astype(int)
input_['Type Code'] = input_['Type Code'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            df_temp, page = pd.DataFrame(), 1
            while True:
                b = 0
                while True:
                    b += 1

                    try:
                        resp = requests.get(f'''https://ecatalog.smpcorp.com/V2/STD/api/part/partsearch?filter=-All-&filterType=n&searchType=p&imageSize=80&start={page*96-96}&limit=96&sort=3&catFilter={input_.loc[a, 'Type Code']}&yearFilter=-All-&makeFilter=-All-&modelFilter=-All-&engineFilter=-All-&attrCodeFilter=-All-&attrValueFilter=-All-&plkEngineMakeFilter=-All-&plkEngineModelFilter=-All-&plkEngineDispFilter=-All-''',
                                             headers=get_header(),
                                             proxies=get_proxy(),
                                             timeout=(10, 10))


                        if resp.status_code == 200:
                            list_dict = resp.json()

                            if list_dict:
                                break
                    except:
                        continue
            
                # = = = = = = = = = = = = = = =

                if page == 1:
                    count = list_dict[0]['maxRows']

                # = = = = = = = = = = = = = = =

                df_temp_temp = pd.DataFrame({'Type No': input_.loc[a, 'Type No'],
                                             'Type': input_.loc[a, 'Type'],
                                             'Type Code': input_.loc[a, 'Type Code'],
                                             'SKU Count': count,
                                             'No': [dict_['rowId'] for dict_ in list_dict],
                                             'Part Number': [dict_['basePart'] for dict_ in list_dict]})
            
                df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                if work.qsize() % 100 == 0 and work.qsize() != 0:
                    clear_output()
                
                print(f'''[尝试次数：{b}] - {(a + 1) / len(input_) * 100:.2f}% - {input_.loc[a, 'Type No']}.{input_.loc[a, 'Type']} > Page {page}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')
            
                # = = = = = = = = = = = = = = =
            
                page += 1

                # = = = = = = = = = = = = = = =

                if len(df_temp) == count:
                    break

            # = = = = = = = = = = = = = = =

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            df_temp = pd.DataFrame([{'Type No': input_.loc[a, 'Type No'],
                                     'Type': input_.loc[a, 'Type'],
                                     'Type Code': input_.loc[a, 'Type Code']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(10):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Type No', 'No'],
                                                ascending=[True, True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./2.part_number-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['Type No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./part_number_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[尝试次数：14] - 93.65% - 1474.Switch - Overdrive Kick-Down > Page 1
[剩余数量：100] - [当前时间：17:17:04]

[尝试次数：1] - 93.71% - 1475.Switch - Parking Brake > Page 1
[剩余数量：99] - [当前时间：17:17:07]

[尝试次数：18] - 93.71% - 1475.Switch - Parking Brake > Page 2
[剩余数量：99] - [当前时间：17:17:11]

[尝试次数：29] - 93.77% - 1476.Switch - Pedal Height Adjustment > Page 1
[剩余数量：98] - [当前时间：17:17:15]

[尝试次数：2] - 93.84% - 1477.Switch - Ported Vacuum > Page 1
[剩余数量：97] - [当前时间：17:17:17]

[尝试次数：12] - 93.90% - 1478.Switch - Power Door Lock > Page 1
[剩余数量：96] - [当前时间：17:17:21]

[尝试次数：35] - 93.90% - 1478.Switch - Power Door Lock > Page 2
[剩余数量：96] - [当前时间：17:17:24]

[尝试次数：6] - 93.90% - 1478.Switch - Power Door Lock > Page 3
[剩余数量：96] - [当前时间：17:17:33]

[尝试次数：13] - 93.90% - 1478.Switch - Power Door Lock > Page 4
[剩余数量：96] - [当前时间：17:17:38]

[尝试次数：6] - 93.90% - 1478.Switch - Power Door Lock > Page 5
[剩余数量：96] - [当前时间：17:17:39]

[尝试次数：3] - 93.96% - 1479.Switch - Power Seat > Page 1
[剩余数量：95] - [当前时间：17:17:41]

[尝试次数：1] - 93.96% - 1479